In [1]:
import pickle
from pathlib import Path
import sys

import pandas as pd
from sklearn.metrics import accuracy_score

rank = 0 
#sys.path[0] = f'{Path().resolve().parents[rank]}' # mymoduleが上の階層にある場合rankを調整してコメント解除
from module.mymodule import PipeLine, df_copy, train_or_test, test_eval
from module.kayano import age_categolize, stSlope_categolize, cholesterol_mean

### データセットの用意

In [2]:
df = pd.read_csv('./data/train.csv')
df_x = df.iloc[:, :-1]
df_y = df.iloc[:, -1]

### パイプライン候補の準備

In [3]:
def pipe_1(df, train_flg, split_kwrg):
    pipe = PipeLine()
    pipe.train_flg = train_flg
    pipe(df)
    pipe.standard_scaler()
    pack = train_or_test(pipe, train_flg, split_kwrg)
    return pack

def pipe_2(df, train_flg, split_kwrg):
    pipe = PipeLine()
    pipe.train_flg = train_flg
    pipe(df)
    pipe.df_num = df_copy(pipe.df_num, stSlope_categolize(pipe.df_cat), 'ST_Slope')
    pipe.standard_scaler()
    pack = train_or_test(pipe, train_flg, split_kwrg)
    return pack

def pipe_3(df, train_flg, split_kwrg):
    pipe = PipeLine()
    pipe.train_flg = train_flg
    pipe.train_flg = train_flg
    pipe(df)
    pipe.df_num = cholesterol_mean(pipe.df_num)
    pipe.standard_scaler()
    pack = train_or_test(pipe, train_flg, split_kwrg)
    return pack

### データセットの作成

In [4]:
split_kwrg = {}
train_flg = False

# パイプライをセット
pipe_lines = [pipe_1, pipe_2, pipe_3]
data_set = {pipe.__name__: pipe(df_x, train_flg, split_kwrg).values for pipe in pipe_lines}

### モデルのロード

In [5]:
with open('./retrained_model.pkl', 'rb') as f:
    train_models = pickle.load(f)

### モデルの評価と選定

In [6]:
evaluations = test_eval(train_models, pipe_lines, data_set, df_y)
evaluations.sort_values('XGBClassifier', ascending=False)

,XGBClassifier,SVC
pipe_3,0.978193,0.839564
pipe_1,0.937695,0.856698
pipe_2,0.912773,0.892523


In [7]:
evaluations.sort_values('SVC', ascending=False)

,XGBClassifier,SVC
pipe_2,0.912773,0.892523
pipe_1,0.937695,0.856698
pipe_3,0.978193,0.839564
